In [1]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers.adapters.composition import Stack
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoConfig, AutoAdapterModel
from transformers import AdapterConfig
from transformers import TrainingArguments, AdapterTrainer
from datasets import concatenate_datasets
import numpy as np
from transformers import EvalPrediction

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

def get_data(lang, data):
    data = data[data["language"] == lang]
    return data

def get_languages(data):
    # data = pd.read_csv("data/train.tsv",sep="\t")
    return data["language"].unique()

def encode_batch(examples):
    all_encoded = {"input_ids": [], "attention_mask": []}
    for premise, hypothesis in (zip(examples["premise"], examples["hypothesis"])):
        # encode separately to maintain CLS and SEP tokens
        encoded = tokenizer.batch_encode_plus(
            [premise, hypothesis],
            add_special_tokens = True,
            pad_to_max_length = True,
            max_length = 100 ,
            return_tensors = 'pt',
            truncation = True
            )

        all_encoded["input_ids"].append(encoded["input_ids"].flatten())
        all_encoded["attention_mask"].append(encoded["attention_mask"].flatten())

    return all_encoded

def preprocess_dataset(dataset):
  # Encode the input data
  dataset = dataset.map(encode_batch, batched=True)
  # The transformers model expects the target class column to be named "labels"
  dataset = dataset.rename_column("gold_label", "labels")
  # Transform to pytorch tensors and only output the required columns
  dataset.set_format(columns=["input_ids", "attention_mask", "labels"])
  return dataset
    

In [2]:
def make_dataset(df, lang):
    en_data = get_data(lang, df)
    en_data = en_data.dropna()
    print("Size of data for language", lang, en_data.shape)

    labels = en_data["gold_label"].values
    labels = [0 if label == "entailment" else 1 if label == "neutral" else 2 for label in labels]
    en_data["gold_label"] = labels
    en_data = Dataset.from_pandas(en_data)

    dataset_en = preprocess_dataset(en_data)
    dataset_en = dataset_en.remove_columns(["language", "premise", "hypothesis", "__index_level_0__"])

    return dataset_en

In [11]:
print("Tokenizing data")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

config = AutoConfig.from_pretrained(
        "xlm-roberta-base",
    )
model = AutoAdapterModel.from_pretrained(
    "xlm-roberta-base",
    config=config,
)

# Load the language adapters
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)


# load all language adapters 
lang_adapters = {
    "en" : "en/wiki@ukp",
    "vi" : "vi/wiki@ukp",
    "de" : "de/wiki@ukp",
    "ar" : "ar/wiki@ukp",
    "es" : "es/wiki@ukp",
    "bg" : "bg/wiki@ukp",
    "el" : "el/wiki@ukp",
    "th" : "th/wiki@ukp",
    "ru" : "ru/wiki@ukp",
    "tr" : "tr/wiki@ukp",
    "sw" : "sw/wiki@ukp",
    "ur" : "ur/wiki@ukp",
    "zh" : "zh/wiki@ukp",
    "hi" : "hi/wiki@ukp",
    "fr" : "fr/wiki@ukp",
}

print("Reading data")
df = pd.read_csv("data/train.tsv",sep="\t")
df = df.sample(frac=0.01 , random_state = 42)

# split data into 80 and 20 in each language

languages = df.language.unique()
train_data = None
for lang in languages:
    lang_data = df[df.language == lang]
    if train_data is None:
        train_data = lang_data.sample(frac=0.8, random_state=42)
        test_data = lang_data.drop(train_data.index)
    else:
        temp = lang_data.sample(frac=0.8, random_state=42)
        train_data = pd.concat([train_data, temp])
        test_data = pd.concat([test_data, lang_data.drop(temp.index)])

print("Size of Train data", train_data.shape)
print("Size of Test data", test_data.shape)

# Add a classification head for our target task
model.add_classification_head("nli", num_labels=3)

# Add a new task adapter
model.add_adapter("nli")

languages = get_languages(df)

Tokenizing data


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /Users/kshitijalwadhi/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformer

Reading data


Adding head 'nli' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}, 'use_pooler': False, 'bias': True}.
Adding adapter 'nli'.


Size of Train data (920, 4)
Size of Test data (230, 4)


In [12]:
languages

array(['th', 'en', 'sw', 'hi', 'fr', 'ar', 'de', 'es', 'ur', 'ru', 'el',
       'zh', 'bg', 'tr', 'vi'], dtype=object)

In [13]:
languages = ['en']

In [14]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [15]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
from transformers import EarlyStoppingCallback, IntervalStrategy

In [17]:
acc = {}

for lang in languages:

    en_data_train = make_dataset(train_data, lang)
    en_data_test = make_dataset(test_data, lang)

    print("Loading language adapter for {}".format(lang))
    try:
        model.load_adapter(lang_adapters[lang], config=lang_adapter_config)
        model.train_adapter(["nli"])
        model.active_adapters = Stack(lang, "nli")
    except:
        model.add_adapter(lang_adapters[lang], config = lang_adapter_config)
        model.train_adapter([lang_adapters[lang],"nli"])
        model.save_adapter("adapters_trained/", lang_adapters[lang])
        model.train_adapter(["nli"])
        model.active_adapters = Stack(lang_adapters[lang], "nli")

    training_args = TrainingArguments(
        learning_rate=3e-4,
        num_train_epochs=1,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        logging_steps=100,
        output_dir="./training_output",
        overwrite_output_dir=True,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
        evaluation_strategy = "epoch", # "steps"
        save_strategy = 'epoch',
        # eval_steps = 50, # Evaluation and Save happens every 50 steps,
        metric_for_best_model = 'f1',
        load_best_model_at_end=True
    )

    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=en_data_train,
        eval_dataset = en_data_test,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
    )

    print(f"Training on {lang}")

    trainer.train()    

    # eval_trainer = AdapterTrainer(
    # model=model,
    # args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    # eval_dataset=en_data_test,
    # compute_metrics=compute_accuracy,
    # )

    # acc[lang] = eval_trainer.evaluate()["eval_acc"]
    # print(acc)


model.save_adapter("adapters_trained/", "nli")

Size of data for language en (814, 4)


  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/homebrew/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Size of data for language en (203, 4)


  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 203
  Batch size = 64


  0%|          | 0/4 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./training_output/checkpoint-13
Configuration saved in ./training_output/checkpoint-13/nli/adapter_config.json
Module weights saved in ./training_output/checkpoint-13/nli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-13/nli/head_config.json
Module weights saved in ./training_output/checkpoint-13/nli/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-13/en/adapter_config.json
Module weights saved in ./training_output/checkpoint-13/en/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-13/nli/head_config.json
Module weights saved in ./training_output/checkpoint-13/nli/pytorch_model_head.bin


Tra

{'eval_loss': 1.0963269472122192, 'eval_accuracy': 0.3497536945812808, 'eval_precision': 0.12232764687325584, 'eval_recall': 0.3497536945812808, 'eval_f1': 0.18125921398007985, 'eval_runtime': 13.086, 'eval_samples_per_second': 15.513, 'eval_steps_per_second': 0.306, 'epoch': 1.0}
{'train_runtime': 195.6008, 'train_samples_per_second': 4.162, 'train_steps_per_second': 0.066, 'train_loss': 1.1232525018545299, 'epoch': 1.0}


In [18]:
acc = {}
languages = [ 'th', 'sw', 'hi', 'fr', 'ar', 'de', 'es', 'ur', 'ru', 'el',
       'zh', 'bg', 'tr', 'vi']

for lang in languages:
    try:
      en_data_train = make_dataset(train_data, lang)
      en_data_test = make_dataset(test_data, lang)

      print("Loading language adapter for {}".format(lang))
      try:
          model.load_adapter(lang_adapters[lang], config=lang_adapter_config)
          model.active_adapters = Stack(lang, "nli")
      except:
          model.load_adapter(lang_adapters[lang], config = lang_adapter_config)
          model.active_adapters = Stack(lang_adapters[lang], "nli")

      eval_trainer = AdapterTrainer(
      model=model,
      args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
      eval_dataset=en_data_test,
      compute_metrics=compute_accuracy,
      )

      acc[lang] = eval_trainer.evaluate()["eval_acc"]
      print(acc)
    except:
      print(f"couldn't do it {lang}")

Size of data for language th (8, 4)


  0%|          | 0/1 [00:00<?, ?ba/s]

Size of data for language th (2, 4)


/opt/homebrew/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

...

Loading language adapter for vi


Resolved adapter files at https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/vi/xlm-roberta-base/pfeiffer/vi_relu_2.zip.
https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/vi/xlm-roberta-base/pfeiffer/vi_relu_2.zip not found in cache or force_download set to True, downloading to /Users/kshitijalwadhi/Documents/Github/FewShot-CrossLingual-TransferLearning/~/.cache/torch/adapters/tmpufc5whb3


storing https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/vi/xlm-roberta-base/pfeiffer/vi_relu_2.zip in cache at ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792.9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c
creating metadata file for ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792.9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c
Loading module configuration from ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792-9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c-extracted/adapter_config.json
Adding adapter 'vi'.
Loading module weights from ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792-9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c-extracted/pytorch_adapter.bin
No matching prediction head found in '~/.cache/torch/adapters/dc145cc33cddd26f442238

  0%|          | 0/1 [00:00<?, ?it/s]

{'th': 0.0, 'sw': 0.0, 'hi': 0.3333333333333333, 'ar': 0.0, 'de': 0.0, 'es': 0.3333333333333333, 'ru': 0.0, 'el': 0.5, 'zh': 0.0, 'tr': 0.0, 'vi': 0.0}


In [19]:
acc

{'th': 0.0,
 'sw': 0.0,
 'hi': 0.3333333333333333,
 'ar': 0.0,
 'de': 0.0,
 'es': 0.3333333333333333,
 'ru': 0.0,
 'el': 0.5,
 'zh': 0.0,
 'tr': 0.0,
 'vi': 0.0}

In [20]:
!mkdir saved_model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
model.save_pretrained("saved_model")

Configuration saved in saved_model/config.json
Model weights saved in saved_model/pytorch_model.bin


In [22]:
!mkdir saved_adapters

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
model.save_adapter("saved_adapters/", "nli")

Configuration saved in saved_adapters/adapter_config.json
Module weights saved in saved_adapters/pytorch_adapter.bin
Configuration saved in saved_adapters/head_config.json
Module weights saved in saved_adapters/pytorch_model_head.bin


In [24]:
del model

In [25]:
config = AutoConfig.from_pretrained(
        "xlm-roberta-base",
    )
model = AutoAdapterModel.from_pretrained(
    "saved_model",
    config=config,
)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /Users/kshitijalwadhi/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



In [33]:
# load task adapter
model.load_adapter("saved_adapters", config=lang_adapter_config, model_name="xlm-roberta-base", load_as="nli")

Loading module configuration from saved_adapters/adapter_config.json
Overwriting existing adapter 'nli'.
Loading module weights from saved_adapters/pytorch_adapter.bin
Loading module configuration from saved_adapters/head_config.json
Overwriting existing head 'nli'
Adding head 'nli' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}, 'use_pooler': False, 'bias': True}.
Loading module weights from saved_adapters/pytorch_model_head.bin


'nli'

In [38]:
model.active_adapters = Stack("nli")

In [39]:
model.active_adapters

Stack[nli]

In [44]:
acc = {}
languages = [ 'th', 'sw', 'hi', 'fr', 'ar', 'de', 'es', 'ur', 'ru', 'el',
       'zh', 'bg', 'tr', 'vi']

for lang in languages:
    try:
      en_data_train = make_dataset(train_data, lang)
      en_data_test = make_dataset(test_data, lang)

      print("Loading language adapter for {}".format(lang))
      try:
          model.load_adapter(lang_adapters[lang], config=lang_adapter_config, model_name="xlm-roberta-base")
          model.active_adapters = Stack(lang, "nli")
      except:
          model.load_adapter(lang_adapters[lang], config = lang_adapter_config, model_name="xlm-roberta-base")
          model.active_adapters = Stack(lang_adapters[lang], "nli")

      eval_trainer = AdapterTrainer(
      model=model,
      args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
      eval_dataset=en_data_test,
      compute_metrics=compute_accuracy,
      )

      acc[lang] = eval_trainer.evaluate()["eval_acc"]
      print(acc)
    except:
      print(f"couldn't do it {lang}")

Size of data for language th (8, 4)


  0%|          | 0/1 [00:00<?, ?ba/s]

Size of data for language th (2, 4)


  0%|          | 0/1 [00:00<?, ?ba/s]

Attempting to load adapter from source 'ah'...


...

Attempting to load adapter from source 'ah'...
Found matching adapter at: adapters/ukp/xlm-roberta-base-vi-wiki_pfeiffer.json


Loading language adapter for vi


Resolved adapter files at https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/vi/xlm-roberta-base/pfeiffer/vi_relu_2.zip.
Loading module configuration from ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792-9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c-extracted/adapter_config.json
Adding adapter 'vi'.
Loading module weights from ~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792-9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c-extracted/pytorch_adapter.bin
No matching prediction head found in '~/.cache/torch/adapters/dc145cc33cddd26f442238ba80f71eb4e510ce84ee464579f2311e24f36fb792-9c56b7f5ae57339c2dce0d880efd56b4ad9019ffa2775dc19a36bc6de4bede0c-extracted'
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same 

  0%|          | 0/1 [00:00<?, ?it/s]

{'th': 0.0, 'sw': 0.0, 'hi': 0.3333333333333333, 'ar': 0.0, 'de': 0.0, 'es': 0.3333333333333333, 'ru': 0.0, 'el': 0.5, 'zh': 0.0, 'tr': 0.0, 'vi': 0.0}


In [45]:
acc

{'th': 0.0,
 'sw': 0.0,
 'hi': 0.3333333333333333,
 'ar': 0.0,
 'de': 0.0,
 'es': 0.3333333333333333,
 'ru': 0.0,
 'el': 0.5,
 'zh': 0.0,
 'tr': 0.0,
 'vi': 0.0}